# Assignment 2.1
### Scrape and Analyse

* API [https://beautiful-soup-4.readthedocs.io/en/latest/](https://beautiful-soup-4.readthedocs.io/en/latest/)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Tasks
Scrape data from the website [http://www.nationmaster.com](http://www.nationmaster.com/), convert it into Pandas data frames and use pandas queries to answer the following questions:

#### 1
Get the number of internet users per country, remove all NaN entries and return the top 10 countries with the highest absolute number of internet users.

In [7]:
#use request to get data from URL
res = requests.get("http://www.nationmaster.com/country-info/stats/Media/Internet-users", verify=False)
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df = pd.read_html(str(table)) #returns list of dataframes (one for each table)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nationmaster.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [8]:
df0 = df[0].drop(columns=['GRAPH', 'HISTORY'])

In [9]:
df0 = df0.dropna(subset=['#','COUNTRY','AMOUNT'])

In [10]:
df0 = df0.drop(columns=['#'])

In [11]:
df0

,COUNTRY,AMOUNT,DATE
0,China,389 million,2009
1,United States,245 million,2009
2,Japan,99.18 million,2009
4,Brazil,75.98 million,2009
5,Germany,65.12 million,2010
...,...,...,...
244,Niue,1100,2009
245,"Saint Helena, Ascension, and Tristan da Cunha",900,2009
246,Saint Helena,900,2009
247,Tokelau,800,2008


In [12]:
df0['AMOUNT'] = df0['AMOUNT'].replace({'million': '*1e6'}, regex=True)

In [13]:
df0['AMOUNT'] = df0['AMOUNT'].map(pd.eval)

In [14]:
df0.sort_values(by=['AMOUNT'], ascending=False).head(10)

,COUNTRY,AMOUNT,DATE
0,China,389000000.0,2009
1,United States,245000000.0,2009
2,Japan,99180000.0,2009
4,Brazil,75980000.0,2009
5,Germany,65120000.0,2010
6,India,61340000.0,2009
7,Russia,59700000.0,2010
9,United Kingdom,51440000.0,2009
10,France,44630000.0,2010
11,Nigeria,43990000.0,2009


#### 2
Get the number of internet users per country, remove all NaN entries and return the top 10 countries with the highest number of internet users relative to the populutation. Hint: you need to scrape the population number from another page)

In [21]:
#use request to get data from URL
res = requests.get("http://www.nationmaster.com/country-info/stats/People/Population", verify=False)
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df_pop_list = pd.read_html(str(table)) #returns list of dataframes (one for each table)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nationmaster.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [22]:
df_pop = df_pop_list[0]

In [23]:
df_pop = df_pop.drop(columns=['GRAPH', 'HISTORY'])
df_pop = df_pop.dropna(subset=['#','COUNTRY','AMOUNT'])
df_pop = df_pop.drop(columns=['#'])

In [24]:
df_pop

,COUNTRY,AMOUNT,DATE
0,China,1.35 billion,2013
1,India,1.22 billion,2013
2,European Union,509.04 million,2012
4,United States,316.67 million,2013
5,Indonesia,251.16 million,2013
...,...,...,...
296,Coral Sea Islands,0.0,2013
297,Clipperton Island,0.0,2013
298,Wake Island,0.0,2013
299,Spratly Islands,0.0,2013


In [25]:
df_pop['AMOUNT'] = df_pop['AMOUNT'].replace({'million': '*1e6','billion': '*1e9'}, regex=True).map(pd.eval)

In [26]:
result = pd.merge(df0, df_pop, on="COUNTRY")

In [27]:
result

,COUNTRY,AMOUNT_x,DATE_x,AMOUNT_y,DATE_y
0,China,389000000.0,2009,1.350000e+09,2013
1,United States,245000000.0,2009,3.166700e+08,2013
2,Japan,99180000.0,2009,1.272500e+08,2013
3,Brazil,75980000.0,2009,2.010100e+08,2013
4,Germany,65120000.0,2010,8.115000e+07,2013
...,...,...,...,...,...
213,Niue,1100.0,2009,1.229000e+03,2013
214,"Saint Helena, Ascension, and Tristan da Cunha",900.0,2009,7.754000e+03,2013
215,Saint Helena,900.0,2009,7.700000e+03,2011
216,Tokelau,800.0,2008,1.353000e+03,2013


In [28]:
result['relative'] = result.apply(lambda x: x['AMOUNT_x'] / x['AMOUNT_y'], axis=1)

In [29]:
result

,COUNTRY,AMOUNT_x,DATE_x,AMOUNT_y,DATE_y,relative
0,China,389000000.0,2009,1.350000e+09,2013,0.288148
1,United States,245000000.0,2009,3.166700e+08,2013,0.773676
2,Japan,99180000.0,2009,1.272500e+08,2013,0.779411
3,Brazil,75980000.0,2009,2.010100e+08,2013,0.377991
4,Germany,65120000.0,2010,8.115000e+07,2013,0.802465
...,...,...,...,...,...,...
213,Niue,1100.0,2009,1.229000e+03,2013,0.895037
214,"Saint Helena, Ascension, and Tristan da Cunha",900.0,2009,7.754000e+03,2013,0.116069
215,Saint Helena,900.0,2009,7.700000e+03,2011,0.116883
216,Tokelau,800.0,2008,1.353000e+03,2013,0.591279


In [30]:
result.sort_values(by=['relative'], ascending=False).head(10)

,COUNTRY,AMOUNT_x,DATE_x,AMOUNT_y,DATE_y,relative
128,Iceland,301600.0,2010,315281.0,2013,0.956607
54,Norway,4430000.0,2010,4720000.0,2013,0.938559
208,Falkland Islands (Islas Malvinas),2900.0,2009,3140.0,2008,0.923567
33,Sweden,8400000.0,2010,9120000.0,2013,0.921053
213,Niue,1100.0,2009,1229.0,2013,0.895037
27,Netherlands,14870000.0,2010,16810000.0,2013,0.884593
148,Saint Lucia,142900.0,2009,162781.0,2013,0.877867
47,Denmark,4750000.0,2010,5560000.0,2013,0.854317
51,Finland,4480000.0,2010,5270000.0,2013,0.850095
120,Luxembourg,424500.0,2010,514862.0,2013,0.824493


#### 3
Compute the correlation between the crime rate (murders per 100k) and the education level. Compare this to the correlation of crime rate and poverty (relative BIP). Hint: use pandas build in correlation function: [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)

In [31]:
#use request to get data from URL
res = requests.get("https://www.nationmaster.com/country-info/stats/Crime/Violent-crime/Murder-rate", verify=False)
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df_murder_list = pd.read_html(str(table)) #returns list of dataframes (one for each table)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nationmaster.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [32]:
df_murder = df_murder_list[0]

In [34]:
#use request to get data from URL
res = requests.get("https://www.nationmaster.com/country-info/stats/Education/High-school-enrolment-rate", verify=False)
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df_edu_list = pd.read_html(str(table)) #returns list of dataframes (one for each table)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.nationmaster.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [35]:
df_edu = df_edu_list[0]

In [36]:
df_murder = df_murder.drop(columns=['GRAPH', 'HISTORY','DATE'])
df_murder = df_murder.dropna(subset=['#','COUNTRY','AMOUNT'])
df_murder = df_murder.drop(columns=['#'])

df_edu = df_edu.drop(columns=['GRAPH', 'HISTORY','DATE'])
df_edu = df_edu.dropna(subset=['#','COUNTRY','AMOUNT'])
df_edu = df_edu.drop(columns=['#'])

In [37]:
df_murder = df_murder.rename(columns={"AMOUNT": "murder-100k"})
df_edu = df_edu.rename(columns={"AMOUNT": "highschool-rate"})

df_murder_edu = pd.merge(df_murder, df_edu, on="COUNTRY")

In [38]:
df_murder_edu

,COUNTRY,murder-100k,highschool-rate
0,Brazil,40974.0,81.93
1,India,40752.0,85.13
2,Mexico,25757.0,93.82
3,Ethiopia,20239.0,88.48
4,Indonesia,18963.0,89.65
...,...,...,...
156,Samoa,2.0,96.29
157,Brunei,2.0,99.44
158,Liechtenstein,1.0,98.55
159,Iceland,1.0,99.83


In [39]:
df_murder_edu.corr()

<ipython-input-39-220a70a2eccc>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_murder_edu.corr()


,murder-100k,highschool-rate
murder-100k,1.000000,-0.059328
highschool-rate,-0.059328,1.000000


### REST API
#### Using data from [https://www.energidataservice.dk](https://www.energidataservice.dk)

In [40]:
import pandas as pd
import requests
from pandas import json_normalize

In [53]:
#get data from an open energy data service provider
url = 'https://api.energidataservice.dk/dataset/NordpoolMarket?offset=0&sort=HourUTC%20DESC&timezone=dk'

response = requests.get(url)
dictr = response.json() #parse json to dict
recs = dictr['records']
df = json_normalize(recs) #flatten json files into data frame
df.head()

,HourUTC,HourDK,PriceArea,SpotPurchase,SpotSale,SpotPriceDKK,ElbasAveragePriceDKK,ElbasMaxPriceDKK,ElbasMinPriceDKK,SpotPriceEUR,ElbasAveragePriceEUR,ElbasMaxPriceEUR,ElbasMinPriceEUR
0,2021-04-30T21:00:00,2021-04-30T23:00:00,DK1,0.0,0.0,512.020020,NaN,NaN,NaN,68.860001,NaN,NaN,NaN
1,2021-04-30T21:00:00,2021-04-30T23:00:00,DK2,776.0,0.0,512.020020,NaN,NaN,NaN,68.860001,NaN,NaN,NaN
2,2021-04-30T20:00:00,2021-04-30T22:00:00,DK1,0.0,0.0,559.390015,472.706512,488.470825,456.496033,75.230003,63.570000,65.690002,61.389999
3,2021-04-30T20:00:00,2021-04-30T22:00:00,DK2,762.0,0.0,559.390015,484.009247,489.214447,479.398926,75.230003,65.089996,65.790001,64.470001
4,2021-04-30T19:00:00,2021-04-30T21:00:00,DK1,0.0,0.0,585.260010,542.604919,557.105103,526.022644,78.709999,72.970001,74.919998,70.739998


#### 4
Compute overview statistics (mean, variance, quantiles, counts,...) for all variables. Hint: there is a single pandas call to get this ...

In [54]:
df.describe()

,SpotPurchase,SpotSale,SpotPriceDKK,ElbasAveragePriceDKK,ElbasMaxPriceDKK,ElbasMinPriceDKK,SpotPriceEUR,ElbasAveragePriceEUR,ElbasMaxPriceEUR,ElbasMinPriceEUR
count,100.000000,100.000000,100.000000,98.000000,98.000000,98.000000,100.000000,98.000000,98.000000,98.000000
mean,304.425999,317.358000,438.767702,442.248511,474.502543,416.679296,59.008200,59.473980,63.811530,56.035408
std,308.401567,472.829590,99.140276,88.715198,89.318835,88.467752,13.333099,11.930500,12.011677,11.897223
min,0.000000,0.000000,257.279999,294.019440,356.928009,261.747192,34.599998,39.540001,48.000000,35.200001
25%,0.000000,0.000000,366.502495,373.937851,402.008759,353.618972,49.290001,50.287499,54.062499,47.555000
50%,277.549988,48.550002,407.139999,407.121002,457.313995,380.425766,54.755001,54.750000,61.500000,51.159999
75%,585.000000,559.425003,520.570007,512.042938,540.690155,465.419243,70.010002,68.859997,72.712497,62.590001
max,780.000000,1409.400024,631.510010,631.018982,739.882019,621.723938,84.930000,84.860001,99.500000,83.610001


#### 5
Compute the average ***SpotSale*** by each day.

In [55]:
df['Date'] = pd.to_datetime(df['HourUTC'])
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

In [56]:
df.groupby('Date')['SpotSale'].mean()

Date
2021-04-28    574.512499
2021-04-29    451.195834
2021-04-30    124.597726
Name: SpotSale, dtype: float64

#### 6
Compute the day with the highest variance in ***SpotPurchase***

In [57]:
df.groupby('Date')['SpotPurchase'].var().idxmax()

'2021-04-30'